In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [2]:
def T(a_arr, alpha_arr, d_arr, theta_arr):
    pose = np.identity(4)
    for i in range(len(theta_arr)):
        a, alpha, d, theta = a_arr[i], alpha_arr[i], d_arr[i], theta_arr[i]
        rot = np.matrix([
            [np.cos(theta), -np.sin(theta), 0, 0],
            [np.sin(theta),  np.cos(theta), 0, 0],
            [0            ,              0, 1, d],
            [0            ,              0, 0, 1],
        ])
        trans = np.matrix([
            [1,             0,              0, a],
            [0, np.cos(alpha), -np.sin(alpha), 0],
            [0, np.sin(alpha),  np.cos(alpha), 0],
            [0,             0,              0, 1],
        ])
        pose = np.matmul(pose, np.matmul(rot, trans))
    return pose

In [3]:
def z(i_minus_1, theta_arr, alpha_arr): # returns z_{i-1}.
    #z_0 = np.array([[0,0,1]]).T
    if i_minus_1==0:
        return np.array([0,0,1]).T
    R = np.identity(3)
    for x in range(1, i_minus_1+1): # x in theta_arr:
        theta, alpha = theta_arr[x-1], alpha_arr[x-1]
        R_z = np.matrix([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [            0,              0, 1]
        ])
        R_x = np.matrix([
            [1,             0,              0],
            [0, np.cos(alpha), -np.sin(alpha)],
            [0, np.sin(alpha),  np.cos(alpha)],
        ])
        R = np.matmul(R, np.matmul(R_z, R_x))
    return np.asarray(R[:,2].T)[0].T # third column

In [4]:
z(2, [np.pi/2, -np.pi/2], [np.pi/4, np.pi/2])

array([ 2.53632657e-17, -1.00000000e+00,  9.81389106e-33])

In [5]:
def p(i_minus_1, a_arr, alpha_arr, d_arr, theta_arr):
    a, alpha, d, theta = a_arr[:i_minus_1+1], alpha_arr[:i_minus_1+1], d_arr[:i_minus_1+1], theta_arr[:i_minus_1+1]
    #a, alpha, d, theta = a_arr[i_minus_1], alpha_arr[i_minus_1], d_arr[i_minus_1], theta_arr[i_minus_1]
    pose = T(a, alpha, d, theta)
    return np.asarray(pose[:3,-1].T)[0].T

In [6]:
def jacobian(a_arr, alpha_arr, d_arr, theta_arr):
    I = len(theta_arr)
    p_e = p(I-1, a_arr, alpha_arr, d_arr, theta_arr)
    J = np.zeros(shape=(6,I))
    for i in range(I):
        z_i = z(i, theta_arr, alpha_arr)
        diff = p_e - p(i, a_arr, alpha_arr, d_arr, theta_arr)
        cp = np.cross(z_i.T, diff.T)
        col = np.concatenate((cp, z_i))
        J[:,i] = col
    return J

In [7]:
a_arr = [0, 0, 0, 0.0825, -0.0825, 0, 0.088, 0]
alpha_arr = [0, -np.pi/2, np.pi/2, np.pi/2, -np.pi/2, np.pi/2, np.pi/2, 0]
d_arr = [0.333, 0, 0.316, 0, 0.384, 0, 0, 0.107]
theta_arr = [0.1, 0.2, 0.3, -0.4, 0.5, 0.6, 0.7]
J = jacobian(a_arr, alpha_arr, d_arr, theta_arr)

### Iterative IK

In [8]:
e = np.zeros(shape=(6,))
T_k = np.matrix([ # = T_i
    [0.515, 0.481, 0.709, -.0815],
    [0.476, -.849, 0.230, -.0409],
    [0.713, 0.219, -.667,  0.399],
    [    0,     0,      0,      1]
])
T_d = np.matrix([
    [-.781, -.474, 0.407, -.0206],
    [-.220, -.818, 0.531,  -.147],
    [-.585, 0.325, -.743,  0.620],
    [    0,     0,     0,       1]
])
q_i = [0.1, 0.2, 0.3, -0.4, 0.5, 0.6, 0.7]

In [17]:
q_arr = []
for k in range(100): # TODO convergence via norm
    theta_arr = q_i # TODO whack?
    T_k = T(a_arr, alpha_arr, d_arr, theta_arr)
    J_k = jacobian(a_arr, alpha_arr, d_arr, theta_arr)
    
    e_p = T_d[:3,-1] - T_k[:3,-1]
    e_o = np.cross(T_k[:3,0].T, T_d[:3,0].T) +\
            np.cross(T_k[:3,1].T, T_d[:3,1].T) +\
            np.cross(T_k[:3,2].T, T_d[:3,2].T)    
    e = np.concatenate((np.asarray(e_p.T)[0], np.asarray(e_o)[0])) # TODO check this
    
    J_inv = J_k.T
    delta_q_k = np.matmul(J_inv, e)
    q_i = q_i + delta_q_k
    q_arr.append(q_i)

In [19]:
with open('q.csv', 'w') as f:
    for row in q_arr:
        n_row = ['{:.3f}'.format(n) for n in row.tolist()]
        f.write('%s\n' % ','.join(n_row))